In [1]:

# import sys
# import os

# # # Ensure the project root is correctly in sys.path
# # project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
# # if project_root not in sys.path:
#     sys.path.insert(0, project_root)

# # # Correct data paths
# # artists_path = os.path.join(project_root, "data", "artists.csv")
# # conflicts_path = os.path.join(project_root, "data", "conflicts.csv")


In [2]:
import sys

sys.path.append("..")

In [3]:
from musical_festival_lineup.musical_festival_lineup import MusicalFestivalSolution
from musical_festival_lineup.musical_festival_data import MusicalFestivalData
from musical_festival_lineup.utils import visualize_musical_lineup
import numpy as np
import time

In [4]:
%load_ext autoreload
%autoreload 2

### 1. Structuring the problem

The  objective  is  to  design  the  optimal  festival  lineup  by  scheduling  artists  across  stages  and 
time  slots while:
- Maximizing  prime  slot  popularity
- Ensuring  genre  diversity  among  stages 
- Minimizing  fan conflicts  at each time slot. 

__`Details`__

- **Artists**: An  artist is  characterized  by  a  popularity  score  (from  0  to  100) a genre  and  fan  base.
    <br>This information with be on __`artists`__ from musical_festival_data

- **Conflicts**:   with  other  artists  (score  from  0  to  1).  Conflicts  arise  when  artists  with  overlapping 
fan  bases  perform  at  the  same  time  but  on  different  stages,  which  can  negatively  impact 
attendance.
    <br>This information will be on __`conflits`__ from musical_festival_data


### 2. Defining a  solution/individual

Each  individual  represents  a  festival  lineup,  specifying  which  artist  plays  on  which  stage  and 
at  which  time  slot. 
These  are  the  constraints  : 
- Each artist is assigned to exactly one stage and slot 
- All time slots start and end at the same time. 
- All stages have the same amount of slots
- These artists should be distributed across  **5 stages  with  7 time slots each**

__`Important`__ 
<br>Any  lineup  where  an  artist  is  assigned  to  **multiple  time  slots  or 
left  unassigned** is  not  part  of  the  search  space  and  is  not  considered  a  solution.  It  is 
forbidden to generate such an arrangement during evolution. 


__`we propose a  having  a solution as`__:

- A string of 5 (stages)*7(slots)= 35 numbers of **1** to **35**
- As for the genetic algorithms, we should have string of the same size, we will be appending a 0 to 1 digit numbers.
- So the solution will be a string of 35 * 2 (two character)= so will have i string of 70

Here a individual of our problem:

In [5]:
data = MusicalFestivalData()

In [6]:
lineup=MusicalFestivalSolution(data=data, repr="2406083423021829103033012519173112162614150532070320000422092728131121")
",".join([ str(i) for i in lineup.repr])

'24,6,8,34,23,2,18,29,10,30,33,1,25,19,17,31,12,16,26,14,15,5,32,7,3,20,0,4,22,9,27,28,13,11,21'

In [7]:
# Python code to show a random initialized solution
lineup=MusicalFestivalSolution(data=data)
",".join([ str(i) for i in lineup.repr])

'18,34,12,31,26,28,24,30,23,29,22,13,6,16,27,5,15,4,8,19,2,17,25,9,20,7,33,14,32,1,21,10,11,0,3'

- Having that, we have to options:
- 1. The first 7 numbers represent the stage 1 , having each of the 7 representing the slots in  order (1 to 7) by the slot?
- 2. Or the first 5 would represent the slot 1 (first time slot) , having the each of the 5 representing the stages in order (1 to 5)?

For that we choose the option 2, because how our fitness will be defined. It is well explained in the session 3, when defining fitness


In [8]:
visualize_musical_lineup("2406083423021829103033012519173112162614150532070320000422092728131121", data.artists)

,Slot,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
0,Slot 1,24: Shadow Cadence|Jazz|66,6: Aurora Skies|Pop|75,8: Crimson Harmony|Classical|20,34: Parallel Dimension|Electronic|58,23: Electric Serpents|Electronic|99
1,Slot 2,2: Velvet Pulse|Jazz|35,18: Velvet Underground|Rock|72,29: Harmonic Dissonance|Classical|96,10: The Wandering Notes|Jazz|84,30: Turbo Vortex|Rock|53
2,Slot 3,33: Cosmic Frequency|Rock|53,1: Solar Flare|Electronic|78,25: Rhythm Alchemy|Jazz|94,19: Astral Tide|Electronic|69,17: Nightfall Sonata|Classical|84
3,Slot 4,31: The Jazz Nomads|Jazz|64,12: Blue Horizon|Pop|51,16: Mystic Rhythms|Classical|78,26: Cloud Nine Collective|Pop|97,14: Synthwave Saints|Rock|94
4,Slot 5,15: Golden Ember|Rock|61,5: Echo Chamber|Electronic|98,32: The Bassline Architects|Hip-Hop|61,7: Static Mirage|Rock|94,3: Neon Reverie|Electronic|100
5,Slot 6,20: The Sonic Drifters|Rock|88,0: Midnight Echo|Rock|75,4: The Silver Owls|Classical|85,22: Quantum Beat|Hip-Hop|96,9: Deep Resonance|Jazz|90
6,Slot 7,27: Hypnotic Echoes|Rock|77,28: The Polyrhythm Syndicate|Jazz|66,13: Lunar Spectrum|Rock|99,11: Phantom Groove|Hip-Hop|47,21: Celestial Voyage|Electronic|95


In [9]:
visualize_musical_lineup(lineup.repr, data.artists)

,Slot,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
0,Slot 1,18: Velvet Underground|Rock|72,34: Parallel Dimension|Electronic|58,12: Blue Horizon|Pop|51,31: The Jazz Nomads|Jazz|64,26: Cloud Nine Collective|Pop|97
1,Slot 2,28: The Polyrhythm Syndicate|Jazz|66,24: Shadow Cadence|Jazz|66,30: Turbo Vortex|Rock|53,23: Electric Serpents|Electronic|99,29: Harmonic Dissonance|Classical|96
2,Slot 3,22: Quantum Beat|Hip-Hop|96,13: Lunar Spectrum|Rock|99,6: Aurora Skies|Pop|75,16: Mystic Rhythms|Classical|78,27: Hypnotic Echoes|Rock|77
3,Slot 4,5: Echo Chamber|Electronic|98,15: Golden Ember|Rock|61,4: The Silver Owls|Classical|85,8: Crimson Harmony|Classical|20,19: Astral Tide|Electronic|69
4,Slot 5,2: Velvet Pulse|Jazz|35,17: Nightfall Sonata|Classical|84,25: Rhythm Alchemy|Jazz|94,9: Deep Resonance|Jazz|90,20: The Sonic Drifters|Rock|88
5,Slot 6,7: Static Mirage|Rock|94,33: Cosmic Frequency|Rock|53,14: Synthwave Saints|Rock|94,32: The Bassline Architects|Hip-Hop|61,1: Solar Flare|Electronic|78
6,Slot 7,21: Celestial Voyage|Electronic|95,10: The Wandering Notes|Jazz|84,11: Phantom Groove|Hip-Hop|47,0: Midnight Echo|Rock|75,3: Neon Reverie|Electronic|100


- Get artists of slot

In [10]:
lineup._get_slot_repr_list(slot=6)

[21, 10, 11, 0, 3]

### 3. Defining fitness

" The  quality  of  a  festival  lineup  is  determined  by  balancing  __`three  equally`__  important 
objectives** ,  each  contributing  to  an  overall  score.  Since  these  objectives  have  different 
scales,  ** they  must  be  normalized  to  the  same  range  (between  0  and  1)  to  ensure  they 
contribute equally to the final fitness score.
<br>
 These are the objectives: 
 
__`Prime  Slot  Popularity`__  :  The  most  popular  artists  should  be  scheduled  in  the  prime 
slots  (**the  last  time  slot  on  each  stage**).  This  score  is  calculated  by  normalizing  the 
total  popularity  of  artists  in  prime  slots  against  the  maximum  possible  total  popularity 
(e.g. if only most popular artists - score 100 - were scheduled on the prime slot). 

__`Genre  Diversity`__  :  A  diverse  distribution  of  **genres  among  stages  in  each  time  slot**
improves  the  festival  experience.  This  score  is  determined  by  normalizing  the 
number  of  unique  genres  in  each  slot  relative  to  the  maximum  possible  number  of 
unique  genres  (e.g.  if  only  different  genres  were  scheduled  in  that  time  slot)  .  Then 
you take the average among time slots. 

__`Conflict  Penalty`__  : ** Fan  conflicts  occur  when  artists  with  overlapping  audiences 
perform  at  the  same  time  on  different  stages**. 
 This  score  is  calculated  by  normalizing 
the  total  conflict  value  in  each  slot  against  the  worst  possible  conflict  scenario  (e.g. 
where  all  artists  with  maximum  conflict  are  scheduled  together).  Then  you  take  the 
average  among  time  slots.  Since  conflicts  negatively  impact  the  lineup,  this  is  a 
penalization  score. 
"

__`Prime  Slot  Popularity`__  

1.  Maximum  possible  total  popularity, having into account that There is 5 stages, so we eill be summing the 5(NUM STAGES) largest

In [11]:
data.max_popularity_in_prime_slot

493

2. Total  popularity  of  artists: <br>
For that, it was defined a function that receives artists Ids and returns the popularity for that specifics Ids. <br>
Lets test it!



In [12]:
#Summing popularity of all artists
data.get_sum_popularity(list(range(14)))

1031

In [13]:
#Summing popularity of [1,2,3,4]
data.get_sum_popularity([1,2,3,4])

298

3. Normalizing the popularity against the max number of popularity

In [14]:
lineup._get_popularity_normalized([1,2,3,4])

0.6044624746450304

__`Genre  Diversity`__  : 

1. Get maximum number of genres per slot, can not be greater than the total number of stages, which is 5

In [15]:
#Genre  Diversity
data.max_distinct_genre_per_slot

5

2. Number of distinct genres : <br>
For that, it was defined a function that receives artists Ids and returns distinct genres in that list of specifics Ids. <br>
Lets test it!


In [16]:
data.get_count_distinct_genres(artists_ids_list=[0,1,10,2])

3

3. Normalizing the genres against the max number of distinct genres

In [17]:
lineup._get_genre_diversity_normalized(artists_ids_list=[0,1,10])

0.6

__`Penalty Conflict`__  : 

1. Get maximum possible conflift, the  worst  possible  conflict  scenario, so for that we get the top K worst case scenerio. <br>
 What is K?
 Is the numbers os conflits in a slot, which is the combination of 5 elements when grouping in two so, we will have C5,2, tthat is equal to 10.
 So will will be choosing the top 10 worst conflits


In [18]:
data.max_worst_conflit_per_slot

np.float64(10.0)

2. The  Total  conflict  value  in  each  slot : <br>
For that, it was defined a function that receives artists Ids and sum of all conflicts in the list. <br>
Lets test it!


In [19]:
data.get_sum_conflicts(artists_ids_list=[3,1,10])

np.float64(1.8)

In [20]:
data.get_sum_conflicts(artists_ids_list=[32, 29, 25, 13, 30])

np.float64(5.75)

In [21]:
data.get_sum_conflicts(artists_ids_list=[2,10])

np.float64(0.9)

3. Normalizing the conflicts against the worst conflit scenario

In [22]:
lineup._get_conflicts_normalized(artists_ids_list=[3,1,10])

np.float64(0.18)

In [23]:
lineup._get_conflicts_normalized(artists_ids_list=[32, 29, 25, 13, 30])

np.float64(0.575)

##### 3.2 Other important functions in the solution

In [24]:
lineup.repr

[18,
 34,
 12,
 31,
 26,
 28,
 24,
 30,
 23,
 29,
 22,
 13,
 6,
 16,
 27,
 5,
 15,
 4,
 8,
 19,
 2,
 17,
 25,
 9,
 20,
 7,
 33,
 14,
 32,
 1,
 21,
 10,
 11,
 0,
 3]

- Get the list of artists Ids in a Slot, Slot is a parameter

In [25]:
lineup._get_slot_repr_list(1)

[28, 24, 30, 23, 29]

In [26]:
lineup._get_slot_repr_list(0)

[18, 34, 12, 31, 26]

##### 3.3 Fitness

In [27]:
#Random 1
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [25, 10, 30, 0, 27]
Slot 0: Conflitcs: 0.6, genres: 0.4, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.6)], List of genres: [0.4], Popularity of the prime slot: 0
Slot of artists: [3, 34, 32, 18, 15]
Slot 1: Conflitcs: 0.6300000000000001, genres: 0.6, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.6), np.float64(0.6300000000000001)], List of genres: [0.4, 0.6], Popularity of the prime slot: 0
Slot of artists: [24, 31, 5, 23, 1]
Slot 2: Conflitcs: 0.49000000000000005, genres: 0.4, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.6), np.float64(0.6300000000000001), np.float64(0.49000000000000005)], List of genres: [0.4, 0.6, 0.4], Popularity of the prime slot: 0
Slot of artists: [29, 21, 11, 4, 12]
Slot 3: Conflitcs: 0.28, genres: 0.8, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.6), np.float64(0.6300000000000001), np.float64(0.49000000000000005), np.float64(0.28)], List of genres: [0.4, 0.6, 0.4, 0.8], Popularity of t

np.float64(0.8630802665893945)

In [28]:
#Random 2
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [19, 34, 22, 5, 16]
Slot 0: Conflitcs: 0.48500000000000004, genres: 0.6, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.48500000000000004)], List of genres: [0.6], Popularity of the prime slot: 0
Slot of artists: [23, 0, 7, 4, 32]
Slot 1: Conflitcs: 0.56, genres: 0.8, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.48500000000000004), np.float64(0.56)], List of genres: [0.6, 0.8], Popularity of the prime slot: 0
Slot of artists: [11, 10, 26, 30, 1]
Slot 2: Conflitcs: 0.45500000000000007, genres: 1.0, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.48500000000000004), np.float64(0.56), np.float64(0.45500000000000007)], List of genres: [0.6, 0.8, 1.0], Popularity of the prime slot: 0
Slot of artists: [18, 13, 9, 6, 29]
Slot 3: Conflitcs: 0.36499999999999994, genres: 0.8, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.48500000000000004), np.float64(0.56), np.float64(0.45500000000000007), np.float64(0.36499999999999994)], L

np.float64(0.7694668212112429)

In [29]:
#Random 3
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [2, 13, 18, 4, 26]
Slot 0: Conflitcs: 0.755, genres: 0.8, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.755)], List of genres: [0.8], Popularity of the prime slot: 0
Slot of artists: [25, 33, 11, 29, 15]
Slot 1: Conflitcs: 0.4600000000000001, genres: 0.8, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.755), np.float64(0.4600000000000001)], List of genres: [0.8, 0.8], Popularity of the prime slot: 0
Slot of artists: [1, 27, 16, 8, 10]
Slot 2: Conflitcs: 0.32, genres: 0.8, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.755), np.float64(0.4600000000000001), np.float64(0.32)], List of genres: [0.8, 0.8, 0.8], Popularity of the prime slot: 0
Slot of artists: [0, 32, 28, 3, 24]
Slot 3: Conflitcs: 0.41, genres: 0.8, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.755), np.float64(0.4600000000000001), np.float64(0.32), np.float64(0.41)], List of genres: [0.8, 0.8, 0.8, 0.8], Popularity of the prime slot: 0
Slot of artists: [3

np.float64(1.2467487684729066)

### Hill-Climbing

In [31]:
from library.algorithms.hill_climbing import hill_climbing
from musical_festival_lineup.musical_festival_lineup_hill_climbing import MusicalFestivalHillClimbingSolution

In [32]:
lineup=MusicalFestivalHillClimbingSolution(data=data)

In [33]:
solution = hill_climbing(lineup, maximization=True, max_iter=1000, verbose=True)
print(solution.fitness())

Current solution: [26, 33, 6, 23, 14, 32, 4, 29, 5, 27, 0, 24, 7, 22, 13, 31, 25, 17, 15, 3, 10, 9, 12, 18, 34, 21, 16, 2, 8, 28, 20, 1, 30, 11, 19] with fitness 0.8602274702984641
Neighbor: [26, 33, 6, 5, 14, 32, 4, 29, 23, 27, 0, 24, 7, 22, 13, 31, 25, 17, 15, 3, 10, 9, 12, 18, 34, 21, 16, 2, 8, 28, 20, 1, 30, 11, 19] with fitness 0.8395131845841785
Neighbor: [24, 33, 6, 23, 14, 32, 4, 29, 5, 27, 0, 26, 7, 22, 13, 31, 25, 17, 15, 3, 10, 9, 12, 18, 34, 21, 16, 2, 8, 28, 20, 1, 30, 11, 19] with fitness 0.8759417560127499
Neighbor: [26, 33, 6, 23, 14, 32, 4, 29, 5, 27, 0, 24, 7, 22, 13, 31, 25, 17, 15, 18, 10, 9, 12, 3, 34, 21, 16, 2, 8, 28, 20, 1, 30, 11, 19] with fitness 0.7937988988698929
Neighbor: [26, 33, 6, 23, 14, 32, 4, 29, 3, 27, 0, 24, 7, 22, 13, 31, 25, 17, 15, 5, 10, 9, 12, 18, 34, 21, 16, 2, 8, 28, 20, 1, 30, 11, 19] with fitness 0.8873703274413213
Neighbor: [26, 33, 6, 23, 14, 32, 4, 29, 5, 27, 8, 24, 7, 22, 13, 31, 25, 17, 15, 3, 10, 9, 12, 18, 34, 21, 16, 2, 0, 28, 20, 1

In [34]:
visualize_musical_lineup(solution.repr, data.artists)

,Slot,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
0,Slot 1,4: The Silver Owls|Classical|85,28: The Polyrhythm Syndicate|Jazz|66,6: Aurora Skies|Pop|75,30: Turbo Vortex|Rock|53,15: Golden Ember|Rock|61
1,Slot 2,32: The Bassline Architects|Hip-Hop|61,10: The Wandering Notes|Jazz|84,2: Velvet Pulse|Jazz|35,5: Echo Chamber|Electronic|98,14: Synthwave Saints|Rock|94
2,Slot 3,8: Crimson Harmony|Classical|20,1: Solar Flare|Electronic|78,7: Static Mirage|Rock|94,24: Shadow Cadence|Jazz|66,18: Velvet Underground|Rock|72
3,Slot 4,33: Cosmic Frequency|Rock|53,25: Rhythm Alchemy|Jazz|94,19: Astral Tide|Electronic|69,0: Midnight Echo|Rock|75,3: Neon Reverie|Electronic|100
4,Slot 5,29: Harmonic Dissonance|Classical|96,11: Phantom Groove|Hip-Hop|47,12: Blue Horizon|Pop|51,13: Lunar Spectrum|Rock|99,31: The Jazz Nomads|Jazz|64
5,Slot 6,21: Celestial Voyage|Electronic|95,16: Mystic Rhythms|Classical|78,26: Cloud Nine Collective|Pop|97,27: Hypnotic Echoes|Rock|77,34: Parallel Dimension|Electronic|58
6,Slot 7,20: The Sonic Drifters|Rock|88,22: Quantum Beat|Hip-Hop|96,23: Electric Serpents|Electronic|99,9: Deep Resonance|Jazz|90,17: Nightfall Sonata|Classical|84


### Roulette Wheel